In [78]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.graph_objs as go
from ipywidgets import interact, interact_manual
import matplotlib.pyplot as plt
import plotly.express as px
%matplotlib inline
import ipywidgets as widgets
from IPython.display import display
from wordcloud import WordCloud
import nltk
from googletrans import Translator
from textblob import TextBlob
from nltk.corpus import stopwords
import re
from ipywidgets import Dropdown

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\naqvi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [69]:
df = pd.read_csv('Final_Tweets_with_Region.csv')
df

,Unnamed: 0,company_id,twitter_handle,date_tweeted,user_tweeted,tweet,likes_count,retweet_count,reply_count,processed_tweet,english_tweet,sentiment,company_name,Region
0,0,1,@i2dtechnologies,2017-01-01 09:02:48+00:00,i2dtechnologies,Happy New Year! #2017 #newyear https://t.co/b...,1,1,0,happy new year 2017 newyear https co bab2qtlmid,happy new year 2017 newyear https co bab2qtlmid,Positive,i2d Technologies.,Attica
1,1,1,@i2dtechnologies,2017-01-01 09:02:48+00:00,i2dtechnologies,Happy New Year! #2017 #newyear https://t.co/b...,1,1,0,happy new year 2017 newyear https co bab2qtlmid,happy new year 2017 newyear https co bab2qtlmid,Positive,i2d Technologies.,Attica
2,2,1,@i2dtechnologies,2017-01-01 09:02:48+00:00,i2dtechnologies,Happy New Year! #2017 #newyear https://t.co/b...,1,1,0,happy new year 2017 newyear https co bab2qtlmid,happy new year 2017 newyear https co bab2qtlmid,Positive,i2d Technologies.,Attica
3,3,2,@my100mentors,2023-04-12 15:04:20+00:00,PrincesTrustInt,Delivered by our partners @Bizrupt_Lab and @my...,1,0,0,delivered partners bizrupt_lab my100mentors fo...,delivered partners bizrupt_lab my100mentors fo...,Neutral,100mentors,Region of Crete
4,4,2,@my100mentors,2023-03-08 13:58:09+00:00,marketing_gr,Pro Bono mentoring session Î¼Îµ Ï„Î¿ Î•Î»Î»Î·Î...,0,0,0,pro bono mentoring session î¼îµ ï î î î î î î½...,pro bono mentoring session î¼îµ ï î î î î î î½...,Neutral,100mentors,Region of Crete
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200038,200038,602,@ZelusGr,2021-07-16 12:22:49+00:00,ZelusGr,Several leading research firms recently named ...,0,0,0,several leading research firms recently named ...,several leading research firms recently named ...,Positive,ZELUS,Attica
200039,200039,602,@ZelusGr,2021-04-27 09:04:41+00:00,ZelusGr,Proud to lead these activities and of course a...,0,0,0,proud lead activities course aiming best way p...,proud lead activities course aiming best way p...,Positive,ZELUS,Attica
200040,200040,602,@ZelusGr,2021-03-08 16:11:52+00:00,ZelusGr,Happy international women's day! Remember to c...,1,0,0,happy international women day remember claim f...,happy international women day remember claim f...,Positive,ZELUS,Attica
200041,200041,602,@ZelusGr,2020-04-01 09:13:29+00:00,ZelusGr,Succesfully presented the results of our #digi...,1,0,0,succesfully presented results digitalforensics...,succesfully presented results digitalforensics...,Neutral,ZELUS,Attica


In [70]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [23]:
df.describe()

,company_id,likes_count,retweet_count,reply_count
count,200827.000000,200827.000000,200827.000000,200827.000000
mean,266.215439,14.229038,4.268704,0.524023
std,120.822973,2185.389260,894.666910,30.102162
min,1.000000,0.000000,0.000000,0.000000
25%,227.000000,0.000000,0.000000,0.000000
50%,227.000000,0.000000,0.000000,0.000000
75%,243.000000,1.000000,0.000000,0.000000
max,602.000000,640496.000000,279753.000000,9159.000000


In [24]:
def group_sentiments_by_company(df):
    # Group the data by company id and sentiment
    grouped_data = df.groupby(['company_name', 'sentiment'])['sentiment'].count()
    
    # Unstack the data to create a new dataframe with sentiment counts for each company id
    unstacked_data = grouped_data.unstack()
    
    # Fill any missing values with 0
    unstacked_data = unstacked_data.fillna(0)
    
    # Rename the columns for clarity
    unstacked_data.columns = ['Negative', 'Neutral', 'Positive']
    
    # Return the unstacked data as a new dataframe
    return unstacked_data

In [25]:
# Use the group_sentiments_by_company function to group the data by company id
sentiment_counts = group_sentiments_by_company(df)

# Print the sentiment counts for each company id
print(sentiment_counts)

                           Negative  Neutral  Positive
company_name                                          
100mentors                    115.0    811.0    1097.0
AGENSO                         12.0    140.0     112.0
Acromove Inc.                  18.0    102.0     124.0
Advantis.io                    24.0     80.0     154.0
Aether Aeronautics              4.0     16.0       4.0
...                             ...      ...       ...
up2metric                       2.0     15.0      32.0
wappier                        41.0    103.0     258.0
yiufi                           0.0     12.0       0.0
ÎÎ¿ÏƒÎ·Î»ÎµÎ¯Î± Î¤ÏŽÏÎ±       1.0    162.0       1.0
Î»-automata                     3.0      9.0      24.0

[207 rows x 3 columns]


In [26]:
# Define a function to update the dashboard based on the selected company id
def update(company_name):
    # Filter the data to include only the selected company id
    filtered_df = df[df['company_name'] == company_name]
    
    # Group the filtered data by sentiment
    grouped_data = filtered_df.groupby('sentiment').size().reset_index(name='count')
    
    # Create a pie chart of the sentiment data for the selected company id
    fig = go.Figure(data=[go.Pie(labels=grouped_data['sentiment'], values=grouped_data['count'])])
    fig.update_layout(title='Sentiment for Company {}'.format(company_name))
    fig.show()

# Define the range of company ids to display in the dropdown
company_ids = df['company_name'].unique()

# Create a dropdown widget to select the company id
@interact(company_name=company_ids)
def display_sentiment_data(company_name):
    update(company_name)

interactive(children=(Dropdown(description='company_name', options=('i2d Technologies.', '100mentors', 'Acromo…

In [27]:
def group_sentiments_by_year(df):
    # Convert the date column to a pandas datetime object
    df['date_tweeted'] = pd.to_datetime(df['date_tweeted'])
    
    # Add a year column to the dataframe
    df['year'] = df['date_tweeted'].dt.year
    
    # Group the data by company id, year, and sentiment
    grouped_data = df.groupby(['company_name', 'year', 'sentiment'])['sentiment'].count()
    
    # Unstack the data to create a new dataframe with sentiment counts for each company id and year
    unstacked_data = grouped_data.unstack(level=-1, fill_value=0)
    
    # Reset the index to make company id and year columns
    unstacked_data = unstacked_data.reset_index()
    
    # Rename the columns for clarity
    unstacked_data.columns.name = None
    unstacked_data.columns = ['company_name', 'year', 'Negative', 'Neutral', 'Positive']
    
    # Return the unstacked data as a new dataframe
    return unstacked_data

In [28]:
# Use the group_sentiments_by_year function to group the data by company id and year
sentiment_counts_by_year = group_sentiments_by_year(df)

# Print the sentiment counts for each company id and year
print(sentiment_counts_by_year)

                  company_name  year  Negative  Neutral  Positive
0                   100mentors  2014         7       23        32
1                   100mentors  2015         5      111        57
2                   100mentors  2016         4       37        31
3                   100mentors  2017        49      374       350
4                   100mentors  2018         0       80        78
..                         ...   ...       ...      ...       ...
814  ÎÎ¿ÏƒÎ·Î»ÎµÎ¯Î± Î¤ÏŽÏÎ±  2021         0        8         0
815  ÎÎ¿ÏƒÎ·Î»ÎµÎ¯Î± Î¤ÏŽÏÎ±  2022         1       82         0
816  ÎÎ¿ÏƒÎ·Î»ÎµÎ¯Î± Î¤ÏŽÏÎ±  2023         0       27         0
817                Î»-automata  2022         3        4        15
818                Î»-automata  2023         0        5         9

[819 rows x 5 columns]


In [29]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go


# Define a function to group sentiments by company id and year
def group_sentiments_by_year(df):
    # Convert the date column to a pandas datetime object
    df['date_tweeted'] = pd.to_datetime(df['date_tweeted'])
    
    # Add a year column to the dataframe
    df['year'] = df['date_tweeted'].dt.year
    
    # Group the data by company id, year, and sentiment
    grouped_data = df.groupby(['company_name', 'year', 'sentiment'])['sentiment'].count()
    
    # Unstack the data to create a new dataframe with sentiment counts for each company id and year
    unstacked_data = grouped_data.unstack(level=-1, fill_value=0)
    
    # Reset the index to make company id and year columns
    unstacked_data = unstacked_data.reset_index()
    
    # Rename the columns for clarity
    unstacked_data.columns.name = None
    unstacked_data.columns = ['company_name', 'year', 'Negative', 'Neutral', 'Positive']
    
    # Return the unstacked data as a new dataframe
    return unstacked_data

# Group the sentiments by company id and year
sentiment_counts_by_year = group_sentiments_by_year(df)

# Create a line chart showing sentiment counts for each company over time
fig = px.line(sentiment_counts_by_year, x="year", y=["Positive", "Negative", "Neutral"], color="company_name")

# Add chart title and axis labels
fig.update_layout(title='Sentiment Counts by Company and Year', xaxis_title='Year', yaxis_title='Sentiment Count')

# Display the chart
fig.show()


In [30]:
def plot_sentiment(company_name, year):
    df_filtered = df[(df['company_name'] == company_name) & (df['date_tweeted'].dt.year == year)]
    fig = px.line(df_filtered, x='date_tweeted', y='sentiment', title=f'Sentiment Changes for Company {company_name} in {year}')
    fig.show()

In [31]:
company_id_widget = widgets.Dropdown(options=df['company_name'].unique())
year_widget = widgets.IntSlider(min=df['date_tweeted'].dt.year.min(), max=df['date_tweeted'].dt.year.max(), step=1, value=df['date_tweeted'].dt.year.max())

interact(plot_sentiment, company_name=company_id_widget, year=year_widget)

interactive(children=(Dropdown(description='company_name', options=('i2d Technologies.', '100mentors', 'Acromo…

<function __main__.plot_sentiment(company_name, year)>

In [32]:
# Convert the date column to datetime format
df['date'] = pd.to_datetime(df['date_tweeted'])

# Create a dropdown widget for company ID
company_dropdown = widgets.Dropdown(options=list(df['company_name'].unique()), description='Company Name:')

# Create a dropdown widget for year
year_dropdown = widgets.Dropdown(options=list(df['date_tweeted'].dt.year.unique()), description='Year:')

# Create a function to update the pie chart based on dropdown selection
def update_pie_chart(company_name, year):
    # Filter data based on dropdown selection
    filtered_data = df[(df['company_name'] == company_name) & (df['date_tweeted'].dt.year == year)]
    
    # Group data by sentiment category and calculate the count
    sentiment_count = filtered_data.groupby('sentiment')['sentiment'].count()
    
    # Create a pie chart
    plt.pie(sentiment_count, labels=sentiment_count.index, autopct='%1.1f%%')
    plt.title(f"Sentiment distribution for {company_name} in {year}")
    plt.show()

# Create an interactive widget for the pie chart
widgets.interact(update_pie_chart, company_name=company_dropdown, year=year_dropdown)


interactive(children=(Dropdown(description='Company Name:', options=('i2d Technologies.', '100mentors', 'Acrom…

<function __main__.update_pie_chart(company_name, year)>

In [33]:
# Define function to create word cloud
def create_wordcloud(df, sentiment):
    text = ' '.join(df[df['sentiment'] == sentiment]['english_tweet'])
    if text != '':
        wordcloud = WordCloud(width=800, height=400, background_color='white', max_words=50, colormap='Set2').generate(text)
        plt.figure(figsize=(10, 5))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis('off')
        plt.show()
    else:
        print(f"No words found for {sentiment} sentiment.")

In [34]:
# Define function to create dashboard
def create_dashboard(df):
    companies = df['company_name'].unique()
    @interact(company=Dropdown(options=companies, value=companies[0], description='Company Name:'))
    def plot_wordcloud(company):
        company_df = df[df['company_name'] == company]
        print(f"Word cloud for Positive sentiment of company {company}")
        create_wordcloud(company_df, 'Positive')
        print(f"Word cloud for Negative sentiment of company {company}")
        create_wordcloud(company_df, 'Negative')
        print(f"Word cloud for Neutral sentiment of company {company}")
        create_wordcloud(company_df, 'Neutral')

In [35]:
# Create dashboard
create_dashboard(df)

interactive(children=(Dropdown(description='Company Name:', options=('i2d Technologies.', '100mentors', 'Acrom…

In [56]:
import ipywidgets as widgets

# Create the dropdown widget
company_dropdown = widgets.Dropdown(
    options=df['company_name'].unique(),
    description='Company Name:',
    value=df['company_name'].unique()[0],
)

In [57]:
def generate_dashboard(company_name):
    # Filter the dataset based on the selected company ID
    filtered_df = df[df['company_name'] == company_name]
    
    # Calculate the total reach for each social media metric
    total_retweets = filtered_df['retweet_count'].sum()
    total_likes = filtered_df['likes_count'].sum()
    total_replies = filtered_df['reply_count'].sum()
    
    # Create a bar chart to display the social media reach
    fig = px.bar(
        x=['Retweets', 'Likes', 'Replies'],
        y=[total_retweets, total_likes, total_replies],
        labels={'x': 'Social Media Metrics', 'y': 'Total Reach'},
        title=f'Social Media Reach for Company Name {company_name}'
    )
    
    # Show the chart
    fig.show()

In [64]:
output = widgets.Output()

# Define a function to handle the dropdown change event
def on_dropdown_change(change):
    with output:
        output.clear_output()
        generate_dashboard(change.new)

# Attach the on_dropdown_change function to the dropdown widget
company_dropdown.observe(on_dropdown_change, names='value')

# Display the dropdown widget and the output widget
display(company_dropdown)
display(output)

# Generate the initial dashboard
generate_dashboard(company_dropdown.value)

Dropdown(description='Company Name:', index=20, options=('i2d Technologies.', '100mentors', 'Acromove Inc.', '…

Output()

In [65]:
def create_dashboard_retweet():
    # Group the data by company ID and sentiment
    grouped_data = df.groupby(['company_name', 'sentiment']).agg({'retweet_count': 'sum'}).reset_index()

    # Define a function to plot the data
    def plot_data(company_name):
        # Filter the data by company ID
        filtered_data = grouped_data[grouped_data['company_name'] == company_name]

        # Create a bar plot of sentiment vs. retweets
        sns.barplot(x='sentiment', y='retweet_count', data=filtered_data)

        # Set the title and axis labels
        plt.title(f'Social Media Reach for Company Name {company_name}')
        plt.xlabel('Sentiment')
        plt.ylabel('Retweets')

        # Show the plot
        plt.show()

    # Create a dropdown widget for company ID
    company_name_dropdown = widgets.Dropdown(options=df['company_name'].unique(), description='Company Name')

    # Use the interact function to update the plot when the dropdown value changes
    interact(plot_data, company_name=company_name_dropdown)

create_dashboard_retweet()

interactive(children=(Dropdown(description='Company Name', options=('i2d Technologies.', '100mentors', 'Acromo…

In [53]:
def create_dashboard_likes():
    # Group the data by company ID and sentiment
    grouped_data = df.groupby(['company_name', 'sentiment']).agg({'likes_count': 'sum'}).reset_index()

    # Define a function to plot the data
    def plot_data(company_name):
        # Filter the data by company ID
        filtered_data = grouped_data[grouped_data['company_name'] == company_name]

        # Create a bar plot of sentiment vs. retweets
        sns.barplot(x='sentiment', y='likes_count', data=filtered_data)

        # Set the title and axis labels
        plt.title(f'Social Media Reach for Company Name {company_name}')
        plt.xlabel('Sentiment')
        plt.ylabel('Likes')

        # Show the plot
        plt.show()

    # Create a dropdown widget for company ID
    company_name_dropdown = widgets.Dropdown(options=df['company_name'].unique(), description='Company Name')

    # Use the interact function to update the plot when the dropdown value changes
    interact(plot_data, company_name=company_name_dropdown)

create_dashboard_likes()

interactive(children=(Dropdown(description='Company Name', options=('i2d Technologies.', '100mentors', 'Acromo…

In [55]:
def create_dashboard_reply():
    # Group the data by company ID and sentiment
    grouped_data = df.groupby(['company_name', 'sentiment']).agg({'reply_count': 'sum'}).reset_index()

    # Define a function to plot the data
    def plot_data(company_name):
        # Filter the data by company ID
        filtered_data = grouped_data[grouped_data['company_name'] == company_name]

        # Create a bar plot of sentiment vs. retweets
        sns.barplot(x='sentiment', y='reply_count', data=filtered_data)

        # Set the title and axis labels
        plt.title(f'Social Media Reach for Company Name {company_name}')
        plt.xlabel('Sentiment')
        plt.ylabel('Replies')

        # Show the plot
        plt.show()

    # Create a dropdown widget for company ID
    company_name_dropdown = widgets.Dropdown(options=df['company_name'].unique(), description='Company Name')

    # Use the interact function to update the plot when the dropdown value changes
    interact(plot_data, company_name=company_name_dropdown)

create_dashboard_reply()

interactive(children=(Dropdown(description='Company Name', options=('i2d Technologies.', '100mentors', 'Acromo…

In [86]:

# Define a function to update the dashboard based on the selected region
def update_dashboard(Region):
    # Filter the data by the selected region
    df_region = df[df['Region']==Region]
    # Group the data by company and sentiment, and count the number of positive sentiments
    df_sentiments = df_region.groupby(['company_name', 'sentiment']).size().unstack().fillna(0)['Positive']
    # Display the results
    display(df_sentiments)

# Define the dropdown menu
dropdown_region = widgets.Dropdown(
    options=list(df['Region'].unique()),
    description='Region:'
)

# Call the update_dashboard function whenever the dropdown value changes
dropdown_region.observe(lambda change: update_dashboard(change.new), names='value')

# Display the dropdown and the initial dashboard
display(dropdown_region)
update_dashboard(df['Region'].unique()[0])


Dropdown(description='Region:', options=('Attica', ' Region of Crete', 'Central Macedonia', 'South Aegean', 'P…

company_name
AGENSO                       112.0
Acromove Inc.                124.0
Advantis.io                  154.0
Agritrack SA                   2.0
Agroknow                     288.0
                             ...  
up2metric                     32.0
wappier                      258.0
yiufi                          0.0
ÎÎ¿ÏƒÎ·Î»ÎµÎ¯Î± Î¤ÏŽÏÎ±      1.0
Î»-automata                   24.0
Name: Positive, Length: 138, dtype: float64

company_name
Allcancode             40.0
Pandrosia              11.0
Refarm Leisure Farm    22.0
SEEMS PC               21.0
Workadu                28.0
Name: Positive, dtype: float64

company_name
AGENSO                       112.0
Acromove Inc.                124.0
Advantis.io                  154.0
Agritrack SA                   2.0
Agroknow                     288.0
                             ...  
up2metric                     32.0
wappier                      258.0
yiufi                          0.0
ÎÎ¿ÏƒÎ·Î»ÎµÎ¯Î± Î¤ÏŽÏÎ±      1.0
Î»-automata                   24.0
Name: Positive, Length: 138, dtype: float64

company_name
Caerleon Comp     481.0
WeAreTravelers      0.0
Name: Positive, dtype: float64

company_name
AGENSO                       112.0
Acromove Inc.                124.0
Advantis.io                  154.0
Agritrack SA                   2.0
Agroknow                     288.0
                             ...  
up2metric                     32.0
wappier                      258.0
yiufi                          0.0
ÎÎ¿ÏƒÎ·Î»ÎµÎ¯Î± Î¤ÏŽÏÎ±      1.0
Î»-automata                   24.0
Name: Positive, Length: 138, dtype: float64

In [121]:
grouped_data = df.groupby(["Region", "sentiment"])["company_name"].count().reset_index()
@interact
def plot(region=list(grouped_data["Region"].unique())):
    filtered_data = grouped_data[grouped_data["Region"] == region]
    fig = px.bar(filtered_data, x="sentiment", y="company_name", color="sentiment")
    positive_sentiments = filtered_data[filtered_data["sentiment"] == "Positive"]["company_name"].values[0]
    negative_sentiments = filtered_data[filtered_data["sentiment"] == "Negative"]["company_name"].values[0]
    neutral_sentiments = filtered_data[filtered_data["sentiment"] == "Neutral"]["company_name"].values[0]
    print("Companies with Positive Sentiments in {} Region: {}".format(region, positive_sentiments))
    print("Companies with Negative Sentiments in {} Region: {}".format(region, negative_sentiments))
    print("Companies with Neutral Sentiments in {} Region: {}".format(region, neutral_sentiments))
    fig.show()


interactive(children=(Dropdown(description='region', options=(' Region of Crete', 'Attica', 'Central Macedonia…

In [119]:
def generate_pie_chart(Region):
    # Filter the data based on the selected region
    region_data = df[df['Region'] == Region]
    
    # Get the count of companies with positive, negative, and neutral sentiments
    counts = region_data['sentiment'].value_counts()
    
    # Create the pie chart
    plt.pie(counts, labels=counts.index, autopct='%1.1f%%')
    plt.title(f"Sentiment Distribution for {Region}")
    plt.show()

# Create the dropdown widget
region_dropdown = Dropdown(options=df['Region'].unique())

# Call the generate_pie_chart function whenever the selected value in the dropdown changes
interact(generate_pie_chart, Region=region_dropdown)

interactive(children=(Dropdown(description='Region', options=('Attica', ' Region of Crete', 'Central Macedonia…

<function __main__.generate_pie_chart(Region)>